# 3. Taxonomy Classification
## Import Data & Packages

In [1]:
# 1- Import packages
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np
import qiime2 as q2
%matplotlib inline

In [2]:
# 2 - Set working directory 
# os.chdir("/home/jovyan/MicrobiomeAnalysis_TummyTribe/")

# Verify that your wroking directory is the overall project folder (.../MicrobiomeAnalysis_TummyTribe/scripts)
print("Current working directory:", os.getcwd())

Current working directory: /home/jovyan/MicrobiomeAnalysis_TummyTribe/scripts


In [3]:
# 3 - Data directories
data_raw = "../data/raw"
data_dir = "../data/preprocessing"
data_out = "../data/taxonomy"
results_dir = "../results/taxonomy"

# Compare the results for different data bases
## Custom Silva classifier on the V4 region
based on these two QIIME tutorials:
- [Processing, filtering, and evaluating the SILVA database (and other reference sequence data) with RESCRIPt](https://forum.qiime2.org/t/processing-filtering-and-evaluating-the-silva-database-and-other-reference-sequence-data-with-rescript/15494)
- [Using RESCRIPt's 'extract-seq-segments' to extract reference sequences without PCR primer pairs](https://forum.qiime2.org/t/using-rescripts-extract-seq-segments-to-extract-reference-sequences-without-pcr-primer-pairs/23618)
### Preparing the SILVA reference database & train an amplicon-region specific classifier
To reduce computation time and avoid memory errors on a jupyterhub, this step was executed on Euler.

In [9]:
# 1. Download Silva reference (RNA) --> missing permission on euler to download the data directly like this
! qiime rescript get-silva-data \
    --p-version '138.2' \
    --p-target 'SSURef_NR99' \
    --o-silva-sequences  $data_out/silva-138.2-ssu-nr99-rna-seqs.qza \
    --o-silva-taxonomy $data_out/silva-138.2-ssu-nr99-tax.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[RNASequence] to: ../data/taxonomy/silva-138.2-ssu-nr99-rna-seqs.qza
Saved FeatureData[Taxonomy] to: ../data/taxonomy/silva-138.2-ssu-nr99-tax.qza


In [10]:
# 2. Reverse transcribe the RNA into DNA
! qiime rescript reverse-transcribe \
    --i-rna-sequences $data_out/silva-138.2-ssu-nr99-rna-seqs.qza \
    --o-dna-sequences $data_out/silva-138.2-ssu-nr99-seqs.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Sequence] to: ../data/taxonomy/silva-138.2-ssu-nr99-seqs.qza


In [14]:
# 3. Filter out poor quality (e.g. > 4 ambiguous bases or homopolymers of length > 7)
! qiime rescript cull-seqs \
    --i-sequences $data_out/silva-138.2-ssu-nr99-seqs.qza \
    --o-clean-sequences $data_out/silva-138.2-ssu-nr99-seqs-cleaned.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Sequence] to: ../data/taxonomy/silva-138.2-ssu-nr99-seqs-cleaned.qza


In [ ]:
# 4. Filtering sequences by length and taxonomy
! qiime rescript filter-seqs-length-by-taxon \
    --i-sequences $data_out/silva-138.2-ssu-nr99-seqs-cleaned.qza \
    --i-taxonomy $data_out/silva-138.2-ssu-nr99-tax.qza \
    --p-labels Archaea Bacteria Eukaryota \
    --p-min-lens 900 1200 1400 \
    --o-filtered-seqs $data_out/silva-138.2-ssu-nr99-seqs-filt.qza \
    --o-discarded-seqs $data_out/silva-138.2-ssu-nr99-seqs-discard.qza

In [ ]:
# 5. Dereplicate
! qiime rescript dereplicate \
    --i-sequences $data_out/silva-138.2-ssu-nr99-seqs-filt.qza  \
    --i-taxa $data_out/silva-138.2-ssu-nr99-tax.qza \
    --p-mode 'uniq' \
    --o-dereplicated-sequences $data_out/silva-138.2-ssu-nr99-seqs-derep-uniq.qza \
    --o-dereplicated-taxa $data_out/silva-138.2-ssu-nr99-tax-derep-uniq.qza

In [ ]:
# 6. Make amplicon-region specific classifier
! qiime feature-classifier extract-reads \
    --i-sequences $data_out/silva-138.2-ssu-nr99-seqs-derep-uniq.qza \
    --p-f-primer GTGYCAGCMGCCGCGGTAA \
    --p-r-primer GGACTACNVGGGTWTCTAAT \
    --p-n-jobs 2 \
    --p-read-orientation 'forward' \
    --o-reads $data_out/silva-138.2-ssu-nr99-seqs-515f-806r.qza

In [ ]:
# 7. Dereplicate again (could have new replicates in the shorter regions)
! qiime rescript dereplicate \
    --i-sequences $data_out/silva-138.2-ssu-nr99-seqs-515f-806r.qza \
    --i-taxa $data_out/silva-138.2-ssu-nr99-tax-derep-uniq.qza \
    --p-mode 'uniq' \
    --o-dereplicated-sequences $data_out/silva-138.2-ssu-nr99-seqs-515f-806r-uniq.qza \
    --o-dereplicated-taxa $data_out/silva-138.2-ssu-nr99-tax-515f-806r-derep-uniq.qza

In [ ]:
# 8. Train amplicon-region specific classifier
! qiime feature-classifier fit-classifier-naive-bayes \
    --i-reference-reads $data_out/silva-138.2-ssu-nr99-seqs-515f-806r-uniq.qza \
    --i-reference-taxonomy $data_out/silva-138.2-ssu-nr99-tax-515f-806r-derep-uniq.qza \
    --o-classifier $data_out/silva-138.2-ssu-nr99-515f-806r-classifier.qza

## Taxonomy assignment

In [16]:
# 9 - assign taxonomy labels to our ASVs 
! qiime feature-classifier classify-sklearn \
    --i-classifier $data_out/silva-138.2-ssu-nr99-515f-806r-classifier.qza \
    --i-reads $data_dir/dada2_rep_seq.qza \
    --o-classification $data_out/taxonomy.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Taxonomy] to: ../data/taxonomy/taxonomy.qza


In [17]:
# 10 - check if it created the taxonomy artefact
! qiime tools peek $data_out/taxonomy.qza

UUID:        419da88f-f3d0-4c29-9ed6-e09054c80317
Type:        FeatureData[Taxonomy]
Data format: TSVTaxonomyDirectoryFormat


In [18]:
# 11 - create the visualization
! qiime metadata tabulate \
    --m-input-file $data_out/taxonomy.qza \
    --o-visualization $data_out/taxonomy.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: ../data/taxonomy/taxonomy.qzv


In [4]:
Visualization.load(f"{data_out}/taxonomy.qzv")

<visualization: Visualization uuid: e7e3aa92-4120-4982-8863-411d854e8c16>

In [24]:
# 12 - Create interactive taxonomy bar plot
! qiime taxa barplot \
    --i-table $data_dir/dada2_table.qza \
    --i-taxonomy $data_out/taxonomy.qza \
    --m-metadata-file $data_raw/metadata.tsv \
    --o-visualization $data_out/taxa-bar-plots.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: ../data/taxonomy/taxa-bar-plots.qzv


In [5]:
Visualization.load(f"{data_out}/taxa-bar-plots.qzv")

<visualization: Visualization uuid: 60942b38-48af-432f-9604-71292ab03de0>

In [6]:
# 13 - load QIIME 2 artifact files as python objects
taxa = q2.Artifact.load(f'{data_out}/taxonomy.qza')
# view as a `pandas.DataFrame`. Note: Only some Artifact types can be transformed to DataFrames
taxa = taxa.view(pd.DataFrame)

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [7]:
# 14 - Count for each taxonomic level how many ASVs were still identified
ranks = [("d__", "Domain"), ("p__", "Phylum"), ("c__", "Class"), ("o__", "Order"), ("f__", "Family"), ("g__", "Genus"), ("s__", "Species")]
total = len(taxa)
for i, (prefix, name) in enumerate(ranks, start=1):
    count = taxa["Taxon"].str.contains(prefix).sum()
    percent = count / total * 100
    print(f"{i}. {name}: {count} ({percent:.1f}%)")

1. Domain: 3358 (100.0%)
2. Phylum: 3354 (99.9%)
3. Class: 3353 (99.9%)
4. Order: 3345 (99.6%)
5. Family: 3300 (98.3%)
6. Genus: 2940 (87.6%)
7. Species: 2940 (87.6%)


In [8]:
# Ignore this currently
ranks = [
    ("k__", "Kingdom"),
    ("p__", "Phylum"),
    ("c__", "Class"),
    ("o__", "Order"),
    ("f__", "Family"),
    ("g__", "Genus"),
    ("s__", "Species")
]

total = len(taxa)

# Function to check if a rank is assigned (not empty)
def is_assigned(taxon_string, prefix):
    for part in taxon_string.split(";"):
        if part.startswith(prefix) and len(part) > len(prefix):
            return True
    return False

# Print nicely
for i, (prefix, name) in enumerate(ranks, start=1):
    count = taxa["Taxon"].apply(lambda x: is_assigned(x, prefix)).sum()
    percent = count / total * 100
    print(f"{i}. {name}: {count} ({percent:.1f}%)")

1. Kingdom: 0 (0.0%)
2. Phylum: 3354 (99.9%)
3. Class: 3353 (99.9%)
4. Order: 3345 (99.6%)
5. Family: 3300 (98.3%)
6. Genus: 2940 (87.6%)
7. Species: 0 (0.0%)


### Evaluate taxonomic classifier

Masjas classifier (not V4 region specific)

In [40]:
! qiime feature-classifier classify-sklearn \
    --i-classifier $data_out/silva-138.2-ssu-nr99-classifier-masja.qza \
    --i-reads $data_dir/dada2_rep_seq.qza \
    --o-classification $data_out/taxonomy_masja.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Taxonomy] to: ../data/taxonomy/taxonomy_masja.qza


In [ ]:
# It denies permission for me :/
! wget -O $raw_data/silva-138-99-nb-human-stool-weighted-classifier.qza \
https://data.qiime2.org/classifiers/sklearn-1.4.2/silva/silva-138-99-nb-human-stool-weighted-classifier.qza

In [46]:
! qiime rescript evaluate-classifications \
    --i-expected-taxonomies $data_out/silva-138.2-ssu-nr99-tax-515f-806r-derep-uniq.qza\
    --i-observed-taxonomies $data_out/taxonomy.qza \
    --p-labels Julia \
    --o-evaluation $data_out/taxonomy-classifier-evaluation.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Plugin error from rescript:

  Expected and Observed Taxonomies do not match. Expected taxonomy must be a superset of observed taxonomies. Indices of pair 1 do not match.

Debug info has been saved to /tmp/qiime2-q2cli-err-m0i8e8i2.log


In [45]:
! qiime rescript evaluate-classifications \
    --i-expected-taxonomies $data_out/silva-138.2-ssu-nr99-tax-515f-806r-derep-uniq.qza $data_out/silva-138.2-ssu-nr99-tax-515f-806r-derep-uniq.qza\
    --i-observed-taxonomies $data_out/taxonomy.qza $data_out/taxonomy_masja.qza \
    --p-labels Julia Masja \
    --o-evaluation $data_out/taxonomy-classifier-evaluation.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Plugin error from rescript:

  Expected and Observed Taxonomies do not match. Expected taxonomy must be a superset of observed taxonomies. Indices of pair 1 do not match.

Debug info has been saved to /tmp/qiime2-q2cli-err-p72zizw6.log


In [14]:
Visualization.load(f"{data_out}/silva-138.2-ssu-nr99-515f-806r-fit-classifier-evaluation.qzv")

<visualization: Visualization uuid: 025010c9-eafc-45fd-8b4a-6703ac68ff0b>

In [11]:
data_pretrained = "../data/processed-pre_trained"

In [12]:
! qiime rescript evaluate-taxonomy \
  --i-taxonomies $data_out/taxonomy.qza $data_out/taxonomy_masja.qza $data_pretrained/taxonomy_140.qza\
  --p-labels Julia Masja pretrained\
  --o-taxonomy-stats $data_out/taxonomy-evaluation.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: ../data/taxonomy/taxonomy-evaluation.qzv


In [13]:
Visualization.load(f"{data_out}/taxonomy-evaluation.qzv")

<visualization: Visualization uuid: 08092df4-0c5c-40ab-be66-60ace1a82d9f>